In [1]:
import os
from persistent_supervisor import AdsWorker, kill_ads_sim_processes, get_last_loop_number


WORK_PATH = r"E:\\personal_Data\\Document of School\\Uni Stuttgart\\Masterarbeit\\Code\\param_regression\\ADS_Parameter_Fitting\\IV_param_regression\\data_gen_pro"
os.chdir(WORK_PATH)
ADS_PYTHON = r"D:\\ADS\\install\\tools\\python\\python.exe"   # path of ADS built-in Python exe
WORKER_PY  = r"E:\\personal_Data\\Document of School\\Uni Stuttgart\\Masterarbeit\\Code\\param_regression\\ADS_Parameter_Fitting\\IV_param_regression\\data_gen_pro\\runner_ads_once.py"  
SAVE_PATH = r"E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting/IV_param_regression/data/log_bigger_dataset/dataset_symmetric_14param/dataGen_mp_1" # path of generated .h5 files
VERILOGA_DIR = r'E:\\personal_Data\\Document of School\\Uni Stuttgart\\Masterarbeit\\Simulation\\ADS\\ASM_HEMT1_wrk_1_Jia\\veriloga'

# all needed parameters for one run
var_dict = {'VOFF':'-4.1', 'U0':'0.4', 'NS0ACCS':'5e+16', 'NFACTOR':'0', 'ETA0':'1e-9', 'VSAT':'1.9e5', 'VDSCALE':'5', 'CDSCD':'0.05', 'LAMBDA':'0.01', 'MEXPACCD':'2', 'DELTA':'2', 'UA':'3e-9', 'UB':'3e-18', 'U0ACCS':'0.17'}
X_iv_shape = [7,121]
X_gm_shape = [10,71]
Y_shape = [14,1]
args = {
    'workspace_path' : "E:\\personal_Data\\Document of School\\Uni Stuttgart\\Masterarbeit\\Simulation\\ADS\\ASM_HEMT1_wrk_1_Jia",
    'validate_dict' : None,
    'library_name' : "IAF_pGaN_lib",
    'design_name' : "gs66508bv1_Uni_symmetric_mp",
    'instance_name' : "sweep_param_mp",
    'param_range' : {
        'VOFF': (-1.2, 2.6),
        'U0': (0, 2.2),
        'NS0ACCS': (1e15, 1e20),
        'NFACTOR': (0.1, 5),
        'ETA0': (0, 1),
        'VSAT': (5e4, 1e7),
        'VDSCALE': (0.5, 1e6),
        'CDSCD': (1e-5, 0.75),
        'LAMBDA': (0, 0.2),
        'MEXPACCD': (0.05, 12),
        'DELTA': (2, 100),
        'UA': (1e-10, 1e-8),
        'UB': (1e-21, 3e-16),
        'U0ACCS': (5e-2, 0.25)
            # 'TBAR': (1e-10, 1e-5) # one unsure geometry param here in addition
            },
    'vgs_bias_param_sweep_name' : 'Sweep_vgs',
    'vds_bias_param_sweep_name' : 'Sweep_vds',
    'vgs_bias_simulation_name' : 'Output',
    'vds_bias_simulation_name' : 'Transfer',
    'data_shape' : [X_iv_shape, X_gm_shape, Y_shape],
    'iteration_num' : 1000,
    'mp_num' : 30,
    'process_id' : 1,
    'save_path' : SAVE_PATH,
    'verilog_a_dir' : VERILOGA_DIR,
    'new_start': True, 
    'old_stop_index': 0
}

# ADS needed env variables
extra_env = {
    "HPEESOF_DIR": r'D:/ADS/install',
    "PATH": r'D:/ADS/install/bin' + os.environ["PATH"],
    "HOME": r'D:/ADS/dir'
}

# running parameters
timeout_s = 120
total_process = 100


In [ ]:
worker = AdsWorker(
    ads_python_exe=ADS_PYTHON, 
    worker_script=WORKER_PY, 
    extra_env=extra_env,
    log_file=SAVE_PATH + '\\ads_supervisor.log'
    )
worker.start()


for i in range(1,total_process):
    args['process_id'] = i + 1
    args['new_start'] = True
    args['old_stop_index'] = 0
    
    # send one task (with timeout)
    while True:
        try:
            rep = worker.run_task(args, timeout=timeout_s)  
            if not rep.get("active"):
                print("Task failed:", rep)
            else:
                print("Task working well:", rep.get("res"))
            break
        except TimeoutError:
            print("[Jupyter] task timeout -> restart worker & retry")
            worker.restart()
            log_file = SAVE_PATH + '\\ads_supervisor.log'
            loop_num = get_last_loop_number(log_file)
            args['new_start'] = False
            args['old_stop_index'] = int(loop_num)
        
worker.stop(graceful=True)



 [worker connection] {"Connection": true, "msg": "ADS worker ready"}
 [worker] task start (timeout=120s)
 [child-log]  >> Process 1 :: Loop 1/1000 :: used time: 20.29 s
 [child-log]  >> Process 1 :: Loop 2/1000 :: used time: 18.25 s
 [child-log]  >> Process 1 :: Loop 3/1000 :: used time: 20.99 s
 [child-log]  >> Process 1 :: Loop 4/1000 :: used time: 59.09 s
 [child-log]  >> Process 1 :: Loop 5/1000 :: used time: 18.01 s
 [child-log]  >> Process 1 :: Loop 6/1000 :: used time: 17.79 s
 [child-log]  >> Process 1 :: Loop 7/1000 :: used time: 17.26 s
 [child-log]  >> Process 1 :: Loop 8/1000 :: used time: 17.93 s
 [child-log]  >> Process 1 :: Loop 9/1000 :: used time: 17.4 s
 [child-log]  >> Process 1 :: Loop 10/1000 :: used time: 18.09 s
 [child-log]  >> Process 1 :: Loop 11/1000 :: used time: 17.69 s
 [child-log]  >> Process 1 :: Loop 12/1000 :: used time: 20.84 s
 [child-log]  >> Process 1 :: Loop 13/1000 :: used time: 17.25 s
 [child-log]  >> Process 1 :: Loop 14/1000 :: used time: 17.

In [ ]:
# Check data
import h5py
import numpy as np
import matplotlib.pyplot as plt
# TODO: This part is only for testing, to verify the content of saved data
data_save_path = r"E:/personal_Data/Document of School/Uni Stuttgart/Masterarbeit/Code/param_regression/ADS_Parameter_Fitting/IV_param_regression/data/log_bigger_dataset/dataset_symmetric_14param/dataGen_mp_1"
process_id = 1
test_index = 111
output_vds_range = (-3.5,8.5)
output_vgs_range = (1,7)
transfer_vgs_range = (0,7)
transfer_vds_range = (1,10)

# load data
# with h5py.File(f"{data_save_path}\\dataset_process_x.h5", 'r') as f:
with h5py.File(f"{data_save_path}\\dataset_process_{process_id}.h5", 'r') as f:
    print('data structure: ', f.keys())
    print('X_iv shape: ', f['X_iv'].shape)
    print('X_gm shape: ', f['X_gm'].shape)
    print('Y shape: ', f['Y'].shape)

    legend_iv = np.linspace(max(output_vgs_range), min(output_vgs_range), f['X_iv'].shape[1])
    legend_gm = np.linspace(max(transfer_vds_range), min(transfer_vds_range), f['X_gm'].shape[1])
    # create colormap
    cmap_iv = plt.cm.vanimo
    cmap_gm = plt.cm.gnuplot
    norm_iv = plt.Normalize(vmin=legend_iv.min(), vmax=legend_iv.max())
    norm_gm = plt.Normalize(vmin=legend_gm.min(), vmax=legend_gm.max())
    
    test_index -= 1
    fig, ax = plt.subplots(1, 2, figsize=(15, 8)) 
    for i in range(f['X_iv'].shape[1]):
        color = cmap_iv(norm_iv(legend_iv[i]))
        ax[0].plot(np.linspace(min(output_vds_range), max(output_vds_range), f['X_iv'].shape[2]) , f['X_iv'][test_index,i,:].flatten(), label=f"VGS={legend_iv[i]} V", color=color)
        ax[0].legend()
        ax[0].grid()
        ax[0].set_title('output IV test plot')
        ax[0].set_xlabel("VDS (V)")
        ax[0].set_ylabel("IDS (A)")
    for i in range(f['X_gm'].shape[1]):
        color = cmap_gm(norm_gm(legend_gm[i]))
        ax[1].plot(np.linspace(min(transfer_vgs_range), max(transfer_vgs_range), f['X_gm'].shape[2]), f['X_gm'][test_index,i,:], label=f"VDS={legend_gm[i]} V", color=color)
        ax[1].legend()
        ax[1].grid(True)
        ax[1].set_title('transfer IV test plot')
        ax[1].set_xlabel("VGS (V)")
        ax[1].set_ylabel("IDS (A)")

    # add colorbars
    sm_iv = plt.cm.ScalarMappable(cmap=cmap_iv, norm=norm_iv)
    sm_iv.set_array([])  # only for colorbar
    cbar_iv = fig.colorbar(sm_iv, ax=ax[0])
    cbar_iv.set_label("VGS (V)")
    
    sm_gm = plt.cm.ScalarMappable(cmap=cmap_gm, norm=norm_gm)
    sm_gm.set_array([])  # only for colorbar
    cbar_gm = fig.colorbar(sm_gm, ax=ax[1])
    cbar_gm.set_label("VDS (V)")